# Notes
The data generated by this notebook can be used to make the maps in figure 1 and is saved in two locations:
* In the `main` folder under the path pointed to by `FIGURE_PATH`
* In the `d3map` folder in the folder this notebook is in - this makes generating the map from that folder easier.

In [ ]:
%matplotlib inline
import sys
import os
DATA_PATH = os.getenv('DATA_PATH')
CODE_PATH = os.getenv('CODE_PATH')
FIGURE_PATH = os.getenv('FIGURE_PATH')

sys.path.insert(0, os.path.join(CODE_PATH, "tracking_emissions"))

import pandas as pd
import numpy as np
import json
import re

import time

from src.load import EGRID, BA_DATA

from d3map_utils import resetCoords, addDataNodes, data_path, data_path2 


import matplotlib.pyplot as plt

import logging.config
logging.config.fileConfig(os.path.join(CODE_PATH, "tracking_emissions/src/logging.conf"))
logger = logging.getLogger(__name__)

In [ ]:
# Make sure directory exists
os.makedirs(data_path2, exist_ok=True)

In [ ]:
co2 = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_CO2_annual.csv"), variable="CO2")
so2 = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_SO2_annual.csv"), variable="SO2")
nox = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_NOX_annual.csv"), variable="NOX")


In [ ]:
WECC_BAs = [
    "AVA", "AZPS","BANC", "BPAT","CHPD",
    "CISO", "DEAA", "DOPD", "EPE", "GCPD", "GRMA", "GWA",
    "HGMA", "IID", "IPCO", "LDWP", "NEVP", "NWMT",
    "PACE", "PACW", "PGE", "PNM", "PSCO",
    "PSEI", "SCL", "SRP", "TEPC","TIDC",
    "TPWR", "WACM", "WALC", "WAUW", "WWA"]

In [ ]:
# CO2 map
resetCoords()

for poll in ["CO2", "SO2", "NOX"]:
    for variable in ["E", poll]:
        
        variable_scalings = {
            "E": 1e-6,
            "CO2": 1e-6,
            "SO2": 1e-3,
            "NOX": 1e-3,
        }
        polli_scalings = {
            "CO2": 1000,
            "SO2": 1e6,
            "NOX": 1e6,
        }
        legCircleTitles = {
            "E": "Electricity consumption (TWh)",
            "CO2": "Carbon consumption (Mtons)",
            "SO2": "SO2 consumption (ktons)",
            "NOX": "NOX consumption (ktons)"
        }
        legLineTitles = {
            "E": "Electricity trade (TWh)",
            "CO2": "Carbon trade (Mtons)",
            "SO2": "SO2 trade (ktons)",
            "NOX": "NOx trade (ktons)",
        }
        legColorTitles = {
            "CO2": "Consumption-based carbon intensity (kg/MWh)",
            "SO2": "Consumption-based SO2 intensity (ton/MWh)",
            "NOX": "Consumption-based NOX intensity (ton/MWh)",
        }
        titles = {
            "E": "ELECTRICITY",
            "CO2": "CARBON",
            "SO2": "SULFUR DIOXIDE",
            "NOX": "NITROGEN OXIDES"
        }
        units =  {
            "E": "TWh",
            "CO2": "Mtons",
            "SO2": "ktons",
            "NOX": "ktons"
        }

        # Unpack options
        legCircleTitle = legCircleTitles[variable]
        legLineTitle = legLineTitles[variable]
        variable_scaling = variable_scalings[variable]
        unit = units[variable]
        title = titles[variable]
        legColorTitle = legColorTitles[poll]
        polli_scaling = polli_scalings[poll]
        fileNm_out = "graph_%s_%si.json" % (variable, poll)

        poll_data = BA_DATA(fileNm=os.path.join(
                DATA_PATH, "analysis/SEED_%s_annual.csv" % poll),
                            variable="%s" % poll)
        elec = BA_DATA(fileNm=os.path.join(
                DATA_PATH, "analysis/SEED_E_annual.csv"),
                       variable="E")

        if variable == "E":
            variable_data = elec
        else:
            variable_data = poll_data

        # Prepare data
        D = (variable_data.df[variable_data.get_cols(field="D")].values[0]
             * variable_scaling)
        X = (poll_data.df[poll_data.get_cols(field="D")].values[0]
             / elec.df[elec.get_cols(field="D")].values[0])
        

        polli_data = dict(zip(elec.regions, polli_scaling * X))
        elec_data = dict(zip(elec.regions, D))

        baseGraphPath = os.path.join(data_path, "graph2.json")
        with open(baseGraphPath, 'r') as fr:
            graph = json.load(fr)

        data = elec_data
        for ba in elec.regions:
            el = data.pop(ba)
            if el == 0.:
                el = None
            data[ba] = el
        graph = addDataNodes(graph, "E_D", data)
        graph = addDataNodes(graph, "E_D", data, "labels")

        data = polli_data
        for ba in elec.regions:
            el = data.pop(ba)
            if np.isnan(el):
                el = None
            data[ba] = el
        graph = addDataNodes(graph, "CO2_Di", data)
        
        data = {}
        for ba in elec.regions:
            if ba in WECC_BAs:
                data[ba] = "wecc"
            elif ba == "ERCO":
                data[ba] = "erco"
            else:
                data[ba] = "eic"
        graph = addDataNodes(graph, "interconnect", data)

        node_list = [el["shortNm"] for el in graph['nodes']]
        shortNm2ind = {node["shortNm"]:i for i, node in enumerate(graph['nodes'])}


        # Add data for the links
        links = []

        regions = variable_data.regions

        for i in range(len(regions)):
            for j in range(i, len(regions)):
                from_ba = regions[i]
                to_ba = regions[j]
                if ((elec.KEY["ID"] % (from_ba, to_ba) in elec.df.columns)
                        & (elec.KEY["ID"] % (to_ba, from_ba) in elec.df.columns)):
                    elec_transfer = elec.df.loc[
                        :, elec.KEY["ID"] % (from_ba, to_ba)].values[0]
                    
                    if elec_transfer < 0: # Have this be positive
                        from_ba = regions[j]
                        to_ba = regions[i]
                        elec_transfer = elec.df.loc[
                            :, elec.KEY["ID"] % (from_ba, to_ba)].values[0]
                    
                    if variable != "E":
                        elec_transfer *= (polli_data[from_ba] / polli_scaling)
                    links += [{
                            "source": shortNm2ind[from_ba],
                            "target": shortNm2ind[to_ba],
                            "TI": elec_transfer*variable_scaling,
                            "TI_i": polli_data[from_ba]}]

        graph["links"] = links


        graph["meta"] = {
            "colorModeAuto": False,
            "fieldRadius": 'E_D',
            "fieldLineWidth": 'TI',
            "fieldCircle": 'CO2_Di',
            "fieldLineColor": 'TI_i',
            "legColorTitle": legColorTitle,
            "legCircleTitle": legCircleTitle,
            "legLineTitle": legLineTitle,
            "unit": unit,
            "title": title
        }

        graphPath_out = os.path.join(data_path, fileNm_out)
        with open(graphPath_out, 'w') as fw:
            json.dump(graph, fw)
        # Save a local copy
        graphPath_out = os.path.join(data_path2, fileNm_out)
        with open(graphPath_out, 'w') as fw:
            json.dump(graph, fw)

In [ ]:
# Aside: Understanding CA imports of SOx and NOx
so2.df[so2.get_cols(r="CISO", field="NG")+ so2.get_cols(r="CISO", field="D")]

In [ ]:
so2.df.loc[:, [so2.KEY["ID"] % (from_ba, "CISO") for from_ba in so2.get_trade_partners("CISO")]]

In [ ]:
so2.df[so2.get_cols(field="D")].transpose().sort_values(by="2016-01-01 00:00:00", ascending=False)

In [ ]:
nox.df.loc[:, [nox.KEY["ID"] % (from_ba, "CISO") for from_ba in so2.get_trade_partners("CISO")]]

In [ ]:
nox.df[nox.get_cols(r="CISO", field="NG")+ nox.get_cols(r="CISO", field="D")]